# Raspberrypi4 Install

Follow the steps in order to properly install the Raspberrypi:

- Install Raspberry Pi OS using Raspberry Pi Imager (download for windows): https://www.raspberrypi.org/software/
- Run the application and save the image (Raspberry pi OS (32-bit)) to the SD card
- Insert the SD in a RBPi board and connect it to screen
- The system ask you to choose a wifi network and change the timezone, language and password
- Reboot and select Raspberry Pi Configuration to activate the SSH and Camera
- reboot and open a terminal to obtain the IP address using "ifconfig"

You are now ready to connect from your Ubuntu Virtual machine using VS Code or a simple terminal